In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import re

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py)

In [2]:
train = pd.read_csv('./Data/train.csv')

In [3]:
train.info()

train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
train.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
print(train.columns)

Index(['review', 'sentiment'], dtype='object')


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[@#]', '',text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [7]:
train['review'] = train['review'].apply(clean_text)

In [8]:
train.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(train['review'].tolist(), show_progress_bar=True)
print(embeddings)

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py)

In [ ]:
from sklearn.preprocessing import LabelEncoder

y = train['sentiment']
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      4961
           1       0.82      0.82      0.82      5039

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



In [ ]:
import joblib
joblib.dump(clf, 'sentiment_model.pkl')

FileNotFoundError: [Errno 2] No such file or directory: './model/sentiment_model.pkl'

In [ ]:
X_train_half = X_train[:2000]
y_train_half = y_train[:2000]

In [ ]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=5, population_size=20, random_state=42)
tpot.fit(X_train_half, y_train_half)


c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\tpot\tpot_estimator\estimator.py:458: UserWarning:

Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.

Generation:  40%|████      | 2/5 [1:06:18<1:39:27, 1989.20s/it]
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning:

Features [127 223] are constant.

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning:

invalid value encountered in divide



,search_space,<tpot.search_...00260F9608970>
,scorers,['roc_auc_ovr']
,scorers_weights,[1]
,cv,10
,other_objective_functions,[]
,other_objective_functions_weights,[]
,objective_function_names,None
,bigger_is_better,True
,categorical_features,None
,memory,None
,preprocessing,False


In [ ]:
y_pred = tpot.predict(X_test)
print(classification_report(y_test, y_pred))
print(tpot.fitted_pipeline_)


              precision    recall  f1-score   support

           0       0.73      0.81      0.77      4961
           1       0.79      0.70      0.74      5039

    accuracy                           0.75     10000
   macro avg       0.76      0.75      0.75     10000
weighted avg       0.76      0.75      0.75     10000

Pipeline(steps=[('robustscaler',
                 RobustScaler(quantile_range=(0.2475690452454,
                                              0.9671736154307))),
                ('selectfwe', SelectFwe(alpha=0.0012903345738)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                       

In [ ]:
joblib.dump(tpot.fitted_pipeline_, 'tpot_model.pkl')

['tpot_model.pkl']

In [ ]:
for pipeline_str, score in tpot.evaluated_individuals_.items():
    print(pipeline_str)
    print(f"Internal CV score: {score['internal_cv_score']}")
    print("=" * 80)


AttributeError: 'TPOTClassifier' object has no attribute 'evaluated_individuals_'